# Optimal debt payment for friends with ILP and Google OR-Tools

This is the example notebook how to use Google OR-Tools to solve a combinatorial problem. Condidered problem and approach is a bit more difficult than what you can see in tutorial. We will use additional variables to reduce a non-linear problem to a linear one.

### The plot

My friends and I made an awesome trip to Austria. All the time we paid for needs of all: dinners, groccery and stuff

At the end we made a list of all transactions with debts to each other. Then, we need to calculate who need to pay to whom. As we don't like to find easy solutions, it's obvious that we may use Interger Linear Programmin approach to solver the problem!

### The model

*Here we consider only one possible approach to the problem. There are some other and I will describe them later.*

Objective. Out objective is to minimize number of transactions to clear off the debt.

**Variables**

* $k \in [1, \ldots , N]$ - index of person
* $x_{ij} \in [0, \infty]$ - amount of money from person i to j
* $y_{ij} \in \{0, 1 \}$ - if there is a transaction from i to j
* $s_{i}$ - current saldo of the person i

So we use one variable to indicate if transaction is allowed and another one to define amount. 

**Constraints**

* We want that inbound and outbound transactions would be equal to current saldo
* We want that we transfer money ($x > 0$) only if $y = 1$

**The whole ILP (Integer Linear Program)**

$$
\sum_{ij} y_{ij} \rightarrow \min \\
\sum_{i} x_{ik} - \sum_{j} x_{kj} = s_k \> \forall k \\
x_{ij} < y_{ij} \cdot \text{max_amount}
$$

3rd condition may seem wierd. Actually, if we want to leave the problem linear, we **can not** formulate 2nd condition as 

$$\sum_{i} x_{ik} y_{ik} - \sum_{j} x_{kj} y_{ik} = s_k$$

So, the 3rd condition is a kind of workaround. We need to define *max_amount* constant that is also seems artificial.

### Other possible approaches

ILP approach is weak to the problem scaling. It means that if you have more persons it may never be calculated. But from the other side, ILP gives an exact solution.

Alternative to ILP is to use approximate algorithms. There are tons of possibilities but one good idea is to use heuristic for $y$ and solve the problem with fixed $y$ using Constraint Programming (CP) on every step

For example:

* Run a greedy solution. Satisfy saldo of each person one by one
* Set $y_{ij} = 1$ from transactions from greedy solutions
* Loop. 
    * For each $i, j \> : \> y_{ij} = 1$
        * Set $y_{ij} = 0$. 
        * Try CP
        * If solution exists save the change and start new loop iteration
    * If no new solution found - finish the algorighm



In [45]:
# In case you havnt install OR Tools
#!pip install ortools

In [46]:
import numpy as np
import pandas as pd

from ortools.linear_solver import pywraplp

## Data Preprocessing

In [47]:
df_transactions = pd.read_csv('soelden.csv', decimal=',')

In [48]:
df_transactions.head()

,Описание,Дата,Сумма,Кто платил,Саша,Антон,Рома,Валера,Вася,Андрей,Женя,Миша,Проверка
0,Завтрак в отеле,09.02.19,120.0,Саша,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,True
1,Магазин,09.02.19,127.0,Рома,15.88,15.88,15.88,15.88,15.88,15.88,15.88,15.88,True
2,Ужин (Die Alm),09.02.19,208.3,Женя,8.30,31.30,26.90,31.30,15.70,27.50,35.00,32.30,True
3,Ужин чаевые,09.02.19,10.0,Женя,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,True
4,Обед на горе (только 4 человека),10.02.19,62.1,Женя,0.00,0.00,11.70,11.70,0.00,0.00,17.30,21.40,True


In [49]:
persons = df_transactions.columns[4:-1]

In [50]:
costs = (
    df_transactions
    .groupby('Кто платил')['Сумма']
    .sum()
    .to_frame()
    .rename(columns={'Сумма': 'costs'}))

In [51]:
debts = (
    df_transactions[persons]
    .sum()
    .to_frame()
    .rename(columns={0: 'debts'}))

In [52]:
df_result = costs.merge(debts, left_index=True, right_index=True)
df_result['saldo'] = df_result['costs'] - df_result['debts']

In [53]:
df_result

,costs,debts,saldo
Андрей,91.50,238.01,-146.51
Антон,119.98,245.60,-125.62
Валера,247.14,245.70,1.44
Вася,309.88,201.85,108.03
Женя,590.30,346.95,243.35
Миша,342.50,355.47,-12.97
Рома,129.50,267.44,-137.94
Саша,243.10,173.08,70.02


In [61]:
saldo_vals = df_result['saldo'].to_dict()

In [62]:
persons = list(saldo_vals.keys())
saldo_vals

{'Андрей': -146.51,
 'Антон': -125.61999999999999,
 'Валера': 1.4399999999999693,
 'Вася': 108.03000000000003,
 'Женя': 243.3499999999999,
 'Миша': -12.970000000000027,
 'Рома': -137.94,
 'Саша': 70.02000000000001}

In [63]:
person_idx = {person: i for i, person in enumerate(persons)}

person_idx_inv = {v: k for k, v in person_idx.items()}

## The model

In [64]:
solver = pywraplp.Solver('SolveIntegerProblem',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

max_transaction = 1000

allow_vars = {}
amount_vars = {}

saldo_constrains = {}
for person in persons:
    saldo_constrains[person] = [saldo_vals[person]]

objective_expr = solver.Objective()

for person_from in persons:
    allow_vars[person_from] = {}
    amount_vars[person_from] = {}
    
    for person_to in persons:
        if person_from == person_to:
            continue
        allow_vars[person_from][person_to] = solver.BoolVar('y_{}_{}'.format(
            person_idx[person_from], person_idx[person_to]))
        
        amount_vars[person_from][person_to] = solver.NumVar(0.0, solver.infinity(), 'x_{}_{}'.format(
            person_idx[person_from], person_idx[person_to]))
        
        # add to objective if transaction is allowed
        objective_expr.SetCoefficient(allow_vars[person_from][person_to], 1)
        
        saldo_constrains[person_from].append(
            amount_vars[person_from][person_to])
        saldo_constrains[person_to].append(
            -1 * amount_vars[person_from][person_to])
        
        # if y = 0 => x = 0 either
        solver.Add(amount_vars[person_from][person_to] <= allow_vars[person_from][person_to] * max_transaction)
        

for person in persons:
    solver.Add(solver.Sum(saldo_constrains[person]) <= 0.1)
    solver.Add(solver.Sum(saldo_constrains[person]) >= -0.1)

In [65]:
objective_expr.SetMinimization()
solver.Solve()

0

In [67]:
rows = []
for person_from in persons:
    for person_to in persons:
        if person_from == person_to:
            continue
        val = amount_vars[person_from][person_to].solution_value()
        if val > 0:
            rows.append((person_from, person_to, val))
            
df_result = pd.DataFrame(rows, columns=['От кого', 'Кому', 'Сумма'])

In [68]:
df_result

,От кого,Кому,Сумма
0,Андрей,Валера,1.34
1,Андрей,Женя,105.41
2,Андрей,Саша,39.86
3,Антон,Вася,95.26
4,Антон,Саша,30.26
5,Миша,Вася,12.87
6,Рома,Женя,137.84


In [69]:
df_result.groupby('От кого').sum()

,Сумма
От кого,
Андрей,146.61
Антон,125.52
Миша,12.87
Рома,137.84


In [70]:
df_result.groupby('Кому').sum()

,Сумма
Кому,
Валера,1.34
Вася,108.13
Женя,243.25
Саша,70.12
